In [1]:
import cv2
import pandas as pd
import numpy as np
import math

In [2]:
# Read the CSV file
df = pd.read_csv('../Data/2025-Coral/FullData.csv')
df

,Image,x_center,y_center,width,height,Image_angle,x_position,y_position,angle
0,0,49.5,14.5,99,29,179.349024,-1.160411,1.623102,0.0
1,1,49.5,14.5,99,29,178.410085,-1.160411,1.623102,5.0
2,2,49.5,15.0,99,30,177.206378,-1.160411,1.623102,10.0
3,3,50.0,15.0,100,30,176.284296,-1.160411,1.623102,15.0
4,4,50.0,15.0,100,30,175.316745,-1.160411,1.623102,20.0
...,...,...,...,...,...,...,...,...,...
30811,30811,1223.0,13.5,114,27,4.213485,1.039589,1.523102,155.0
30812,30812,1223.0,13.5,114,27,3.273725,1.039589,1.523102,160.0
30813,30813,1223.0,13.0,114,26,2.338709,1.039589,1.523102,165.0
30814,30814,1223.0,13.0,114,26,1.589915,1.039589,1.523102,170.0


When a new image is captured we need to run the same algorithm we ran to create the data

In [3]:
# Works by finding the two longest edges of the contour
# and averaging their angles
# Used for 2025 Coral
def find_angle(contour, img=None, draw=True):
    """
    Given a contour, find the two longest straight lines,
    average their directions, and return the dominant angle in degrees.
    """
    if len(contour) < 2:
        return None

    # 1. Simplify contour (remove small jitter)
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 2. Collect line segments and their angles
    lines = []
    for i in range(len(approx)):
        p1 = approx[i][0]
        p2 = approx[(i + 1) % len(approx)][0]  # wrap around
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        length = math.hypot(dx, dy)
        if length > 2:  # ignore tiny edges
            angle = math.degrees(math.atan2(dy, dx))
            lines.append((length, angle, p1, p2))

    if len(lines) < 2:
        return None

    # 3. Take two longest lines
    lines.sort(reverse=True, key=lambda x: x[0])
    longest = lines[:2]

    # 4. Compute average direction
    # Handle circular mean (avoid averaging 179° and -179° to get 0°)
    angles = [math.radians(l[1]) for l in longest]
    x_mean = np.mean([math.cos(a) for a in angles])
    y_mean = np.mean([math.sin(a) for a in angles])
    avg_angle = math.degrees(math.atan2(y_mean, x_mean))

    avg_angle = (avg_angle + 360 + 90) % 180  # normalize to [0, 180)

    # 5. Optional drawing
    if img is not None and draw:
        for _, angle, p1, p2 in longest:
            cv2.line(img, tuple(p1), tuple(p2), (0, 255, 0), 2)
        # Draw the averaged orientation line at the contour center
        cx, cy = np.mean(approx[:, 0, :], axis=0).astype(int)
        length = 50
        x2 = int(cx + length * math.cos(math.radians(avg_angle)))
        y2 = int(cy + length * math.sin(math.radians(avg_angle)))
        cv2.arrowedLine(img, (cx, cy), (x2, y2), (255, 255, 0), 2, tipLength=0.2)

    return avg_angle

In [6]:
def find_bounding_rect_angle(image, low_bounds, top_bounds):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, low_bounds, top_bounds)
    
    # Find contours
    contours, _ = cv2.findContours(red_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)

    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
    angle = find_angle(contour, image, draw=True)
    
    cv2.imshow("Frame", image)
    cv2.waitKey(1)
    
    return (x + w / 2, y + h / 2), (w, h), angle

Find similar data points inside our data base to find the real world position

In [5]:
def find_matching_rows(df, target, start_tol=2, max_tol=25, step=2):
    """
    Find rows in the dataframe that match the target values within a dynamically
    increasing tolerance. If no rows are found for a very small tolerance, the
    tolerance will be increased until at least one row is found or the maximum
    tolerance is reached.

    Parameters:
        df (pd.DataFrame): DataFrame with the columns.
        target (dict): Target values for each column.
        start_tol (int): Starting tolerance measured in pixels.
        max_tol (int): Maximum allowed tolerance measured in pixels.
        step (int): Increment to increase tolerance on each iteration measured in pixels.

    Returns:
        filtered_df (pd.DataFrame): DataFrame with matching rows.
        used_tol (int): Tolerance at which the matching rows were found.
    """
    
    tolerance = start_tol
    while tolerance <= max_tol:
        # Create mask using np.isclose for each column
        mask = (
            (np.isclose(df['Center_X'], target['Center_X'], atol=tolerance) &
            np.isclose(df['Center_Y'], target['Center_Y'], atol=tolerance) &
            np.isclose(df['Width'], target['Width'], atol=tolerance) &
            np.isclose(df['Height'], target['Height'], atol=tolerance)) &
            (np.isclose(df['Image_angle'], target['Image_angle'], atol=tolerance) |
                np.isclose(df['Image_angle'], (target['Image_angle'] + 180) % 360, atol=tolerance) |
                np.isclose(df['Image_angle'], (target['Image_angle'] - 180) % 360, atol=tolerance))
        )
        filtered_df = df[mask]
        
        # Check if any row is found
        if not filtered_df.empty:
            print(f"Found rows with tolerance: {tolerance}")
            return filtered_df, tolerance
        
        # Increase the tolerance and try again
        tolerance += step

    # No rows found within maximum tolerance
    print("No rows found within the max tolerance.")
    return df.iloc[[]], tolerance  # Return an empty DataFrame

Main function running on the Raspberry-PI

In [ ]:
targetRes = (1280, 720)  # Target resolution for the image

# Capture the image using the connected camera (For testing purposes load an image from disk)
image = cv2.imread("will/be/added/later.jpg")

# Resize the image to the target size
image = cv2.resize(image, targetRes)

# The low bounds, [h, s, v]
low_bounds = np.array([9, 35, 0])
# The top bounds, [h, s, v]
top_bounds = np.array([31, 255, 255])

# Detect the closest game piece in the image, using ml model or a simple color filter
oriented_rect = find_bounding_rect_angle(image, low_bounds, top_bounds)
cv2.destroyAllWindows()
c_x, c_y, w, h, a = oriented_rect

target = {
    'Center_X': c_x,
    'Center_Y': c_y,
    'Width': w,
    'Height': h,
    'Angle': a
}

# Assuming df is the pandas DataFrame
result_df, used_tolerance = find_matching_rows(df, target, start_tol=10, max_tol=40, step=3)
result_df

In [ ]:
coral_position = [result_df['x_position'].mean(), result_df['y_position'].mean()]
print(coral_position)
print(f"Coral angle: {result_df['angle'].mean()} degrees")